In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "Qwen/Qwen1.5-1.8B-Chat"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Qwen2ForCausalLM(
  (model): Qwen2Model(
    (embed_tokens): Embedding(151936, 2048)
    (layers): ModuleList(
      (0-23): 24 x Qwen2DecoderLayer(
        (self_attn): Qwen2Attention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (k_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (v_proj): Linear(in_features=2048, out_features=2048, bias=True)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
        )
        (mlp): Qwen2MLP(
          (gate_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5504, bias=False)
          (down_proj): Linear(in_features=5504, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
        (post_attention_layernorm): Qwen2RMSNorm((2048,), eps=1e-06)
      )
    )
    (norm): Qwen2RMSNorm((2048,), eps=1e-06)
    (rotary_emb): 

In [ ]:
messages = [
    {
        "role": "system",
        "content": (
            "You are a board-certified medical expert. "
            "Answer each question accurately and concisely in two sentences. "
            "Base your answer on evidence-based medicine."
        ),
    },
    {
        "role": "user",
        "content": "Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?"
    },
]

In [ ]:
prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
inputs = tokenizer(prompt, return_tensors="pt").to(device)

In [ ]:
outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    do_sample=False,
    temperature=0.1,
    top_p=0.95
)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


In [ ]:
generated = outputs[0][inputs["input_ids"].shape[1]:]
answer = tokenizer.decode(generated, skip_special_tokens=True).strip()

print("Question:", messages[1]["content"])
print("Answer:", answer)

Question: Do mitochondria play a role in remodelling lace plant leaves during programmed cell death?
Answer: Mitochondria, the powerhouses of cellular energy production, have been implicated in regulating programmed cell death (PCD) processes in plants. Evidence suggests that mitochondria can modulate PCD by controlling the expression of genes involved in autophagy, a process by which cells degrade and recycle their own components to maintain cellular homeostasis. During PCD, mitochondria undergo autophagic degradation, leading to the release of cytosolic organelles such as cytochrome c and reactive oxygen species (ROS). These molecules can trigger apoptosis, or programmed cell death, in neighboring cells through various signaling pathways. In the case of lace plant leaves, it has been reported that mitochondria can regulate autophagy and ROS levels, contributing to the regulation of PCD in these tissues. However, more research is needed to fully understand the role of mitochondria in 

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_name)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    load_in_4bit=True,
    device_map="auto",
    torch_dtype=torch.float16
)

messages = [
    {"role": "user", "content": "What is your favourite condiment?"},
    {"role": "assistant", "content": "Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!"},
    {"role": "user", "content": "Do you have mayonnaise recipes?"}
]

inputs = tokenizer.apply_chat_template(messages, return_tensors="pt").to("cuda")
outputs = model.generate(inputs, max_new_tokens=1000, do_sample=True)
print(tokenizer.decode(outputs[0]))


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
c:\Users\krtgi\anaconda3\Lib\site-packages\bitsandbytes\nn\modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


<s> [INST] What is your favourite condiment? [/INST] Well, I'm quite partial to a good squeeze of fresh lemon juice. It adds just the right amount of zesty flavour to whatever I'm cooking up in the kitchen!</s> [INST] Do you have mayonnaise recipes? [/INST] Yes, I do have a simple and classic mayonnaise recipe for you! Here's what you'll need:

1. 1 cup vegetable oil
2. 1 tablespoon distilled white vinegar or fresh lemon juice
3. 1 tablespoon Dijon mustard
4. 1 teaspoon salt
5. 1 egg yolk
6. 1-2 tablespoons water (optional, for thinning)

Instructions:

1. In a medium-sized bowl, combine the egg yolk, salt, and Dijon mustard. Whisk until they are well blended and the color of the yolk has lightened slightly.
2. Drop in the oil very slowly, in a thin stream, while continuously whisking the mixture.
3. Once all the oil has been incorporated, add the vinegar or lemon juice and whisk to combine.
4. If the mayonnaise is too thick, add a tablespoon of water at a time until it reaches the des